---
Some useful $\LaTeX$ commands are defined in this cell:
$$
\newcommand{\abs}[1]{\left\lvert#1\right\rvert}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\set}[1]{\left\{#1\right\}}
\newcommand{\paren}[1]{\left(#1\right)}
\newcommand{\brack}[1]{\left[#1\right]}
\newcommand{\ip}[2]{\left\langle#1,#2\right\rangle}
\DeclareMathOperator{\span}{span}
\DeclareMathOperator{\fl}{fl}
\abs{x}, \norm{x}, \set{x}, \paren{x}, \brack{x}, \ip{x}{y}, \span, \fl
$$

---

---
# 15.5 Romberg integration
---

In this section, we consider adaptively selecting the **order of the numerical quadrature method**.

**Richardson extrapolation** gave us a systematic way to obtain higher order numerical differentiation methods.

Richardson extrapolation can also be used to obtain higher order numerical quadrature methods.

---

## Richardson extrapolation for numerical integration

It can be shown that the **composite trapezoidal rule** with step-size $h$ 

$$
\fbox{${\displaystyle
\int_a^b f(x) \,dx = \frac{h}{2}\brack{f(a) + 2 \sum_{i=1}^{r-1} f(a + ih) + f(b)} - \frac{f''(\xi)}{12} (b - a) h^2, 
\quad \xi \in [a,b],
}$}
$$

gives an approximation $Q_h$ of $I_f = \int_a^b f(x) \,dx$ such that

$$I_f = Q_h + Kh^2 + \mathcal{O}(h^4).$$

Therefore, dividing the step-size by $2$ gives us:

$$I_f = Q_{h/2} + K(h/2)^2 + \mathcal{O}(h^4).$$

Multiplying this equation by $4$ and subtracting the first equation, we have

$$3I_f = 4 Q_{h/2} - Q_h + \mathcal{O}(h^4).$$

Thus, we obtain 

$$I_f = \frac{4 Q_{h/2} - Q_h}{3} + \mathcal{O}(h^4).$$



---

## Example

Let's use Richardson extrapolation to approximate

$$\int_0^1 e^{-x^2} \,dx = 0.746824132812427\ldots.$$

In [ ]:
using SymPy

t = symbols("t")

f(x) = exp(-x^2)
a, b = 0, 1

inttrue = Float64(integrate(f(t), (t, a, b)))

In [ ]:
using Printf

@printf "%6s %10s %10s %10s %10s %10s %10s\n" "h" "Q1" "Q1err" "Q2" "Q2err" "R" "Rerr"
for k = 0:4
    r = 10^k
    
    h1 = (b - a)/r
    tt1 = a:h1:b
    ft1 = f.(tt1)
    Q1 = (sum(ft1[1:end-1]) + sum(ft1[2:end]))*h1/2
    
    h2 = h1/2
    tt2 = a:h2:b
    ft2 = f.(tt2)
    Q2 = (sum(ft2[1:end-1]) + sum(ft2[2:end]))*h2/2
    
    R = (4Q2 - Q1)/3
    
    @printf "%6.0e %10f %10.2e %10f %10.2e %10f %10.2e\n" h1 Q1 abs(Q1 - inttrue) Q2 abs(Q2 - inttrue) R abs(R - inttrue)
end

We see numerically that the composite trapezoid rule `Q1` is $\mathcal{O}(h^2)$ and that the Richardson extrapolation rule `R` gives us an $\mathcal{O}(h^4)$ method.

---

## A second step of Richardson extrapolation

Let's define

$$R_{1,1} = Q_h = \frac{h}{2}\brack{f(a) + 2 \sum_{i=1}^{r-1} f(a + ih) + f(b)},$$

and

$$
\begin{split}
R_{2,1} = Q_{h/2} 
&= \frac{h/2}{2}\brack{f(a) + 2 \sum_{i=1}^{2r-1} f(a + i(h/2)) + f(b)}\\
&= \frac{1}{2}\brack{R_{1,1} + h \sum_{i=1}^{r} f(a + (2i-1)(h/2))}.\\
\end{split}
$$

Then we let 

$$R_{2,2} = \frac{4 Q_{h/2} - Q_h}{3} = \frac{4 R_{2,1} - R_{1,1}}{3}.$$




Next, we go one step further to obtain

$$
\begin{split}
R_{3,1} = Q_{h/4} 
&= \frac{h/4}{2}\brack{f(a) + 2 \sum_{i=1}^{4r-1} f(a + i(h/4)) + f(b)}\\
&= \frac{1}{2}\brack{R_{2,1} + \frac{h}{2} \sum_{i=1}^{2r} f(a + (2i - 1)(h/4))}.\\
\end{split}
$$

Then we let 

$$R_{3,2} = \frac{4 Q_{h/4} - Q_{h/2}}{3} = \frac{4 R_{3,1} - R_{2,1}}{3}.$$


We can show that

$$
\begin{align}
I_f &= R_{2,2} + Kh^4 + \mathcal{O}(h^6)\\
I_f &= R_{3,2} + K(h/2)^4 + \mathcal{O}(h^6)\\
\end{align}
$$

Thus,

$$
I_f = \frac{16R_{3,2} - R_{2,2}}{15} + \mathcal{O}(h^6).
$$

Let 

$$
R_{3,3} = \frac{16R_{3,2} - R_{2,2}}{15}.
$$

In [ ]:
using SymPy

t = symbols("t")

f(x) = exp(-x^2)
a, b = 0, 1

inttrue = Float64(integrate(f(t), (t, a, b)))

@printf("%6s %10s %10s %10s %10s %10s %10s\n", "h",
        "R11", "R11err", 
        "R22", "R22err",
        "R33", "R33err")
for k = 0:4
    r = 10^k
    
    h1 = (b - a)/r
    tt1 = a:h1:b
    ft1 = f.(tt1)
    R11 = (sum(ft1[1:end-1]) + sum(ft1[2:end]))*h1/2
    
    h2 = h1/2
    tt2 = a:h2:b
    ft2 = f.(tt2)
    R21 = (sum(ft2[1:end-1]) + sum(ft2[2:end]))*h2/2

    h3 = h1/4
    tt3 = a:h3:b
    ft3 = f.(tt3)
    R31 = (sum(ft3[1:end-1]) + sum(ft3[2:end]))*h3/2

    R22 = (4R21 - R11)/3
    R32 = (4R31 - R21)/3
    
    R33 = (16R32 - R22)/15
    
    @printf("%6.0e %10f %10.2e %10f %10.2e %10f %10.2e\n", h1, 
            R11, abs(R11 - inttrue), 
            R22, abs(R22 - inttrue),
            R33, abs(R33 - inttrue))
end

---

## The general algorithm


In [ ]:
using LinearAlgebra

##################################################
function romberg(f, a, b, r, s)
    h = (b - a)/r
    tt = a:h:b
    R = zeros(s,s)
    R[1,1] = h/2*( f(a) + 2sum(f.(tt[2:end-1])) + f(b) )
          
    for j = 2:s
        h = h/2
        tt = a:h:b
        R[j,1] = 1/2*( R[j-1,1] + h*2sum(f.(tt[2:2:end-1])) )
        
        for k = 2:j
            R[j,k] = (4^(k-1)*R[j,k-1] - R[j-1,k-1])/(4^(k-1) - 1)
        end
    end
    
    LowerTriangular(R)
end

##################################################
using SymPy

t = symbols("t")

f(x) = exp(-x^2)
a, b = 0, 1
r, s = 8, 4

inttrue = Float64(integrate(f(t), (t, a, b)))

R = romberg(f, a, b, r, s)

In [ ]:
LowerTriangular(abs.(R .- inttrue))

The **Romberg** table of values is completed one row at a time and gives increasing order of accuracy:

$$
\begin{array}{cccc}
\mathcal{O}(h^2) & \mathcal{O}(h^4) & \mathcal{O}(h^6) & \mathcal{O}(h^8) \\
\hline
R_{1,1} \\
R_{2,1} & R_{2,2} \\
R_{3,1} & R_{3,2} & R_{3,3} \\
R_{4,1} & R_{4,2} & R_{4,3} & R_{4,4} \\
\end{array}
$$

Computing a table with $s$ rows requires $r2^{s} + 1$ function evaluations.

In [ ]:
r*[2^s for s = 1:4] .+ 1

## Note

In order for Romberg integration to be effective, the integrand should be very smooth and the initial $h$ should be quite small.

---